<h1>Python Library Importation</h1>

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
#Word class tag in JSON
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [3]:
#Apply logic for the user need
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
 #classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

48 documents
18 classes ['advice', 'current_mood', 'favorite_color', 'favorite_food', 'favorite_movie', 'games', 'goodbye', 'greeting', 'how_are_you', 'jokes', 'learning', 'motivational_quotes', 'pidgin', 'random_conversation', 'random_fact', 'random_thoughts', 'thanks', 'weather']
82 unique lemmatized words ["'s", 'a', 'advice', 'any', 'appreciate', 'are', 'best', 'bro', 'bye', 'can', 'care', 'chat', 'color', 'do', 'doing', 'eat', 'enjoy', 'evening', 'fact', 'far', 'favorite', 'feel', 'film', 'food', 'for', 'fun', 'game', 'give', 'going', 'good', 'goodbye', 'greeting', 'have', 'hello', 'hey', 'hi', 'how', 'i', 'inspire', 'interesting', 'is', 'it', 'joke', 'later', 'laugh', 'learn', 'like', 'make', 'me', 'mood', 'morning', 'motivation', 'movie', 'need', 'new', 'pizza', 'rain', 'random', 'recommendation', 'see', 'should', 'some', 'something', 'soon', 'take', 'talk', 'teach', 'tell', 'temperature', 'thank', 'thanks', 'the', 'thought', 'to', 'today', 'want', 'we', 'weather', 'what', 'will

In [5]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [6]:
# training set, bag of words for each sentence
training = []
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1 if word match found in current pattern
    for w in words:
        bag.append(1 if w in pattern_words else 0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle our features
random.shuffle(training)
# Separate the data into input (X) and output (y) lists
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])
print("Training data created")


# # create train and test lists. X - patterns, Y - intents
# train_x = list(training[:,0])
# train_y = list(training[:,1])
# print("Training data created")

# Now X and y are numpy arrays with consistent shapes


Training data created


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

Epoch 1/200


c:\users\tega\appdata\local\programs\python\python39\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 1s 3ms/step - loss: 2.8976 - accuracy: 0.0833
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.8363 - accuracy: 0.1042
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.8181 - accuracy: 0.1667
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.7879 - accuracy: 0.2292
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6553 - accuracy: 0.2708
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6556 - accuracy: 0.2708
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6920 - accuracy: 0.2083
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.5748 - accuracy: 0.2292
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 2.4681 - accuracy: 0.2292
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 2.4379 - accuracy: 0.3750
Epoch 11/200
10/10 [=